<a href="https://colab.research.google.com/github/akzholba/RecSysCompetition/blob/bert4rec/bert/Bert4rec_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example of the Bert4Rec training and inference stages
Note that all the given examples can be run without using PySpark, using only Pandas

In [1]:
!pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 37.0 MB/s eta 0:00:00


In [2]:
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import torch

In [3]:
from schema import (
    FeatureHint,
    FeatureInfo,
    FeatureSchema,
    FeatureSource,
    FeatureType,
)

from optimizer_utils import FatOptimizerFactory

from bert4rec import Bert4Rec, Bert4RecModel

from dataset import (
    Bert4RecPredictionDataset,
    Bert4RecTrainingDataset,
    Bert4RecValidationDataset,
    Bert4RecPredictionBatch,
)

import pandas as pd

## Getting a spark session

In [4]:
from data import Dataset, get_spark_session

spark_session = get_spark_session()

## Prepare data
### Load raw movielens-1M interactions, item features and user features.
In the current implementation, the SASRec does not take into account the features of items or users. They are only used to get a complete list of users and items.

In [5]:
# !pip install rs-datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 28.8 MB/s eta 0:00:00


In [6]:
# from rs_datasets import MovieLens

In [7]:
# movielens = MovieLens("1m")
# interactions = movielens.ratings
# user_features = movielens.users
# item_features = movielens.items

In [8]:
# interactions.head()

In [9]:
# user_features.head()

In [10]:
# item_features.head()

In [11]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
le = LabelEncoder()

interactions = pd.read_csv('./data/events.csv')
interactions[interactions['user_id']==0]

item_features = pd.read_csv('./data/item_features.csv')

user_features = pd.read_csv('./data/user_features.csv')
user_features['gender'] = le.fit_transform(user_features['gender'])

Removing duplicates in the timestamp column without changing the original items order where timestamp is the same

In [12]:
interactions["timestamp"] = interactions["timestamp"].astype("int64")
interactions = interactions.sort_values(by="timestamp")
interactions["timestamp"] = interactions.groupby("user_id").cumcount()
interactions

,user_id,item_id,rating,timestamp
0,0,1505,4,0
511292,3433,3022,3,0
511349,3434,3673,2,0
511721,3435,358,1,0
511769,3436,1279,4,0
...,...,...,...,...
336760,2257,3452,3,2051
336761,2257,2749,3,2052
336762,2257,2623,3,2053
336763,2257,1175,4,2054


### Split interactions into the train, validation and test datasets using LastNSplitter

In [13]:
from last_n_splitter import LastNSplitter

splitter = LastNSplitter(
    N=1,
    divide_column="user_id",
    query_column="user_id",
    strategy="interactions",
)

raw_test_events, raw_test_gt = splitter.split(interactions)
raw_validation_events, raw_validation_gt = splitter.split(raw_test_events)
raw_train_events = raw_validation_events

### Prepare FeatureSchema required to create Dataset

In [14]:
def prepare_feature_schema(is_ground_truth: bool) -> FeatureSchema:
    base_features = FeatureSchema(
        [
            FeatureInfo(
                column="user_id",
                feature_hint=FeatureHint.QUERY_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="item_id",
                feature_hint=FeatureHint.ITEM_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
        ]
    )
    if is_ground_truth:
        return base_features

    all_features = base_features + FeatureSchema(
        [
            FeatureInfo(
                column="timestamp",
                feature_type=FeatureType.NUMERICAL,
                feature_hint=FeatureHint.TIMESTAMP,
            ),
        ]
    )
    return all_features

### Create Dataset for the training stage

In [15]:
train_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_train_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the validation stage

In [16]:
validation_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_validation_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
validation_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_validation_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the testing stage

In [17]:
test_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_test_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
test_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_test_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create the tensor schema
A schema shows the correspondence of columns from the source dataset with the internal representation of tensors inside the model

In [18]:
from schema import (
    TensorFeatureSource,
    TensorSchema,
    TensorFeatureInfo)

ITEM_FEATURE_NAME = "item_id_seq"

tensor_schema = TensorSchema(
    TensorFeatureInfo(
        name=ITEM_FEATURE_NAME,
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL,
        feature_sources=[TensorFeatureSource(FeatureSource.INTERACTIONS, train_dataset.feature_schema.item_id_column)],
        feature_hint=FeatureHint.ITEM_ID,
        embedding_dim=300,
    )
)

### Create sequential datasets using SequenceTokenizer
The SequentialDataset internally store data in the form of sequences of items sorted by increasing interaction time (timestamp). A SequenceTokenizer is used to convert to this format. In addition, the SequenceTokenizer encodes all categorical columns from the source dataset and stores mapping inside itself.
SequentialDataset.keep_common_query_ids is used to leave only sequences from the same users

In [19]:
# !pip install pyspark

In [20]:
from sequence_tokenizer import SequenceTokenizer
from sequential_dataset import SequentialDataset

tokenizer = SequenceTokenizer(tensor_schema, allow_collect_to_master=True)
tokenizer.fit(train_dataset)

sequential_train_dataset = tokenizer.transform(train_dataset)

sequential_validation_dataset = tokenizer.transform(validation_dataset)
sequential_validation_gt = tokenizer.transform(validation_gt, [tensor_schema.item_id_feature_name])

sequential_validation_dataset, sequential_validation_gt = SequentialDataset.keep_common_query_ids(
    sequential_validation_dataset, sequential_validation_gt
)

In [21]:
test_query_ids = test_gt.query_ids
test_query_ids_np = tokenizer.query_id_encoder.transform(test_query_ids)["user_id"].values
sequential_test_dataset = tokenizer.transform(test_dataset).filter_by_query_id(test_query_ids_np)

You can get the user and item mapping and inverse mapping as follows

In [22]:
print(tokenizer.query_id_encoder.mapping, tokenizer.query_id_encoder.inverse_mapping)
print(tokenizer.item_id_encoder.mapping, tokenizer.item_id_encoder.inverse_mapping)

{'user_id': {4855: 0, 4065: 1, 3331: 2, 5373: 3, 2032: 4, 5875: 5, 3984: 6, 4062: 7, 5117: 8, 5822: 9, 174: 10, 5188: 11, 595: 12, 2538: 13, 5031: 14, 4765: 15, 1819: 16, 3970: 17, 568: 18, 4007: 19, 2641: 20, 2646: 21, 3839: 22, 3263: 23, 281: 24, 2009: 25, 5836: 26, 1581: 27, 679: 28, 3634: 29, 2401: 30, 2184: 31, 5532: 32, 3638: 33, 4159: 34, 1770: 35, 3754: 36, 637: 37, 1452: 38, 5412: 39, 5345: 40, 3078: 41, 4772: 42, 3484: 43, 1064: 44, 2812: 45, 3120: 46, 4295: 47, 491: 48, 3283: 49, 5595: 50, 622: 51, 4428: 52, 1570: 53, 4561: 54, 3927: 55, 127: 56, 1950: 57, 1877: 58, 2285: 59, 656: 60, 462: 61, 4055: 62, 4477: 63, 2148: 64, 1582: 65, 272: 66, 3556: 67, 883: 68, 5295: 69, 3223: 70, 4070: 71, 3: 72, 5314: 73, 4225: 74, 1341: 75, 5909: 76, 1413: 77, 4463: 78, 3900: 79, 4426: 80, 811: 81, 3491: 82, 5118: 83, 2018: 84, 1308: 85, 4379: 86, 4351: 87, 2995: 88, 3680: 89, 1336: 90, 3758: 91, 1286: 92, 5003: 93, 3574: 94, 1703: 95, 1855: 96, 32: 97, 5901: 98, 5207: 99, 1516: 100, 5457:

## Train model
### Create Bert4Rec model instance and run the training stage using lightning
After each epoch validation metrics are shown. You can change the list of validation metrics in ValidationMetricsCallback
The model is determined to be the best and is saved if the metric updates its maximum during validation (see the ModelCheckpoint)

In [57]:
from validation_callback import ValidationMetricsCallback
from postprocessors import RemoveSeenItems

MAX_SEQ_LEN = 100
BATCH_SIZE = 512
NUM_WORKERS = 4

model = Bert4Rec(
    tensor_schema,
    block_count=2,
    head_count=4,
    max_seq_len=MAX_SEQ_LEN,
    hidden_size=300,
    dropout_rate=0.5,
    optimizer_factory=FatOptimizerFactory(learning_rate=0.001),
)
checkpoint_callback = ModelCheckpoint(
    dirpath=".checkpoints",
    save_top_k=1,
    verbose=True,
    # if you use multiple dataloaders, then add the serial number of the dataloader to the suffix of the metric name.
    # For example,"recall@10/dataloader_idx_0"
    monitor="recall@10",
    mode="max",
)

validation_metrics_callback = ValidationMetricsCallback(
    metrics=["map", "ndcg", "recall"],
    ks=[1, 5, 10, 20],
    item_count=train_dataset.item_count,
    postprocessors=[RemoveSeenItems(sequential_validation_dataset)]
)

csv_logger = CSVLogger(save_dir=".logs/train", name="Bert4Rec_example")

trainer = L.Trainer(
    max_epochs=100,
    callbacks=[checkpoint_callback, validation_metrics_callback],
    logger=csv_logger,
)

train_dataloader = DataLoader(
    dataset=Bert4RecTrainingDataset(
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
)

validation_dataloader = DataLoader(
    dataset=Bert4RecValidationDataset(
        sequential_validation_dataset,
        sequential_validation_gt,
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=validation_dataloader,
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /content/.checkpoints exists and is not empty.
INFO:

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (12) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

k              1        10        20         5
map     0.036093  0.079806  0.087114  0.069498
ndcg    0.036093  0.110394  0.137400  0.085089
recall  0.036093  0.211589  0.319205  0.132616



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 99, global step 1200: 'recall@10' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 99, global step 1200: 'recall@10' was not in top 1
INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


k              1        10        20         5
map     0.036589  0.078300  0.085788  0.067373
ndcg    0.036589  0.108879  0.136390  0.082299
recall  0.036589  0.210430  0.319702  0.127980



The path to the best model is saved inside checkpoint_callback

In [58]:
best_model = Bert4Rec.load_from_checkpoint(checkpoint_callback.best_model_path)

In [88]:
checkpoint_callback.best_model_path

'/content/.checkpoints/epoch=92-step=1116.ckpt'

In [90]:
ls

base_splitter.py          optimizer_utils.py       session_handler.py
Bert4rec_Model.ipynb      output.csv               spark_utils.py
bert4rec.py               postprocessors.py        submission_BERT100epoch.csv
data/                     prediction_callbacks.py  submission.csv
data.py                   __pycache__/             top_10_films.csv
dataset_label_encoder.py  recommendations.csv      torch_metrics_builder.py
dataset.py                sample_data/             torch_sequential_dataset.py
label_encoder.py          schema.py                typesBert.py
last_n_splitter.py        sequence_tokenizer.py    utils.py
model.py                  sequential_dataset.py    validation_callback.py


## Inference stage
### Prepare Dataloader and logger

In [67]:
prediction_dataloader = DataLoader(
    dataset=Bert4RecPredictionDataset(
        sequential_test_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

csv_logger = CSVLogger(save_dir=".logs/test", name="Bert4Rec_example")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


### Run inference
You can get the recommendations in three formats: PySpark DataFrame, Pandas DataFrame, PyTorch tensors. Each of the types corresponds a callback
You can filter the results using postprocessors strategy. For example the RemoveSeenItems postprocessor is filtering out the items that already have been seen in test dataset
You don't need to use all three callbacks. This is shown only for example

Also, you can get user embeddings, that were used to perform predictions, using `get_query_embedding` method inside Bert4RecModel or `QueryEmbeddingsPredictionCallback` for lightning module.

In [68]:
# from replay.metrics import OfflineMetrics, Recall, Precision, MAP, NDCG, HitRate, MRR
# from replay.metrics.torch_metrics_builder import metrics_to_df

from prediction_callbacks import (
    SparkPredictionCallback,
    PandasPredictionCallback,
    TorchPredictionCallback,
    QueryEmbeddingsPredictionCallback,
)

TOPK = [1, 10, 20, 100]

postprocessors = [RemoveSeenItems(sequential_test_dataset)]

spark_prediction_callback = SparkPredictionCallback(
    spark_session=spark_session,
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

pandas_prediction_callback = PandasPredictionCallback(
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

torch_prediction_callback = TorchPredictionCallback(
    top_k=max(TOPK),
    postprocessors=postprocessors,
)

query_embeddings_callback = QueryEmbeddingsPredictionCallback()

trainer = L.Trainer(
    callbacks=[
        spark_prediction_callback,
        pandas_prediction_callback,
        torch_prediction_callback,
        query_embeddings_callback,
    ],
    logger=csv_logger,
    inference_mode=True
)
trainer.predict(best_model, dataloaders=prediction_dataloader, return_predictions=False)

spark_res = spark_prediction_callback.get_result()
pandas_res = pandas_prediction_callback.get_result()
torch_user_ids, torch_item_ids, torch_scores = torch_prediction_callback.get_result()
user_embeddings = query_embeddings_callback.get_result()

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [69]:
spark_res.show()

+-------+-------+-----------------+
|user_id|item_id|            score|
+-------+-------+-----------------+
|    790|   3421|9.312716484069824|
|    790|   1422|8.847206115722656|
|    790|    434|8.750907897949219|
|    790|    708|8.747827529907227|
|    790|   3025| 8.62032413482666|
|    790|   2543|8.587140083312988|
|    790|   1461|  8.5817232131958|
|    790|   2003|8.497234344482422|
|    790|   1332|8.429722785949707|
|    790|   1287|8.354843139648438|
|    790|    566|8.291472434997559|
|    790|    827|8.272329330444336|
|    790|   2138|8.268227577209473|
|    790|   3460|8.233558654785156|
|    790|   2108|8.218437194824219|
|    790|    980|8.189330101013184|
|    790|   2251|8.142723083496094|
|    790|   1250|8.138209342956543|
|    790|   1951|7.928654193878174|
|    790|   1128|7.888408660888672|
+-------+-------+-----------------+
only showing top 20 rows



In [70]:
pandas_res

,user_id,item_id,score
0,790,3421,9.312716
0,790,1422,8.847206
0,790,434,8.750908
0,790,708,8.747828
0,790,3025,8.620324
...,...,...,...
6039,4520,950,4.898004
6039,4520,1906,4.879602
6039,4520,2011,4.870603
6039,4520,960,4.862675


In [71]:
pandas_res.to_csv('output.csv')

In [72]:
print(torch_user_ids[0], torch_item_ids[0], torch_scores[0])

tensor(790) tensor([3421, 1422,  434,  708, 3025, 2543, 1461, 2003, 1332, 1287,  566,  827,
        2138, 3460, 2108,  980, 2251, 1250, 1951, 1128, 1259, 1959, 2143, 1493,
        3699, 3029, 1672, 2146,  874, 2801, 2551, 1329,  904, 1323,  178, 2403,
         516,   74,  446, 3411, 3167, 3393, 1303, 3214,  351, 3481, 1939,  881,
         644, 2456, 1101, 1052, 1014, 3209, 2555, 2593, 2145, 1824, 1905, 2051,
          32, 2397, 3657, 2159, 2687, 1744,  554,  128, 1160, 2604, 1705,  672,
        2702, 1171,  485, 2213,  129, 3103,  994, 3569, 2827,  163, 2925,  411,
        3097, 2275, 1377,  703, 3119, 3007, 3026, 2418, 1920, 1479, 2690, 2339,
        1965,  550, 3056, 1155]) tensor([9.3127, 8.8472, 8.7509, 8.7478, 8.6203, 8.5871, 8.5817, 8.4972, 8.4297,
        8.3548, 8.2915, 8.2723, 8.2682, 8.2336, 8.2184, 8.1893, 8.1427, 8.1382,
        7.9287, 7.8884, 7.8662, 7.8638, 7.8538, 7.8401, 7.7869, 7.7594, 7.7348,
        7.7185, 7.6971, 7.6893, 7.6600, 7.6417, 7.6156, 7.5982, 7.5643, 7.5

Suppose we want to get the recomendations in PySpark format.
Let's get the inverse representation of labels using inverse_transform method.

Note that the reverse representation can only be obtained for PySpark and Pandas formats. When working with PyTorch tensors, the reverse representation must be done manually

In [73]:
recommendations = tokenizer.query_and_item_id_encoder.inverse_transform(spark_res)

In [74]:
recommendations.show()

+-----------------+-------+-------+
|            score|user_id|item_id|
+-----------------+-------+-------+
|8.291472434997559|      0|    566|
|8.138209342956543|      0|   1250|
|7.928654193878174|      0|   1951|
|8.847206115722656|      0|   1422|
|8.272329330444336|      0|    827|
|8.189330101013184|      0|    980|
|8.142723083496094|      0|   2251|
|7.888408660888672|      0|   1128|
|9.312716484069824|      0|   3421|
|8.747827529907227|      0|    708|
|  8.5817232131958|      0|   1461|
|8.354843139648438|      0|   1287|
|8.233558654785156|      0|   3460|
|8.218437194824219|      0|   2108|
| 8.62032413482666|      0|   3025|
|7.866247653961182|      0|   1259|
|8.587140083312988|      0|   2543|
|8.429722785949707|      0|   1332|
|8.268227577209473|      0|   2138|
|8.750907897949219|      0|    434|
+-----------------+-------+-------+
only showing top 20 rows



In [75]:
# Преобразуем Spark DataFrame в Pandas DataFrame
pandas_df2 = recommendations.toPandas()

# Сохраняем результаты в CSV-файл
pandas_df2.to_csv('recommendations.csv', index=False)

### Calculating metrics

In [76]:
init_args = {"query_column": "user_id", "rating_column": "score"}

In [77]:
def get_top_n(user_item_ratings, model_name, n=100):
    '''Функция возвращает топ-n фильмов для каждого пользователя'''

    # Сортируем данные по убыванию предсказанной оценки
    top_n = user_item_ratings.sort_values(model_name, ascending=False)

    # Оставляем только первые n строк для каждого пользователя
    top_n = top_n.groupby('user_id').head(n).reset_index(drop=True)

    return top_n

In [78]:
pandas_df2

,score,user_id,item_id
0,4.368713,3,94
1,4.090495,3,467
2,4.018270,3,1371
3,3.834505,3,2947
4,3.299708,3,1304
...,...,...,...
603995,6.219978,6027,793
603996,6.108172,6027,3684
603997,5.933334,6027,3476
603998,5.698685,6027,3396


In [79]:
top_10_films = get_top_n(pandas_df2, 'score', n=10)[['user_id', 'item_id']]

# Экспортируем результат в CSV-файл
top_10_films.to_csv('top_10_films.csv', index=False)

In [80]:
top_10_films.user_id.nunique()

6040

In [81]:
def format_for_submission(df):
    # Группируем строки по user_id и соединяем item_id через пробел
    submission = (
        df
        .groupby('user_id')['item_id']
        .apply(lambda x: ' '.join(x.astype(str)))
        .reset_index()
    )

    return submission

In [82]:
submission = format_for_submission(top_10_films)
submission

,user_id,item_id
0,0,3421 1422 434 708 3025 2543 1461 2003 1332 1287
1,1,1246 232 350 1459 2476 2730 3656 3101 1822 452
2,2,2774 382 234 2311 1371 1687 1560 221 2428 1781
3,3,3390 3562 20 1814 361 94 1456 1382 467 1371
4,4,983 1160 394 672 755 743 2205 2338 3030 2159
...,...,...
6035,6035,2800 1439 1296 3665 1403 3668 892 3216 1011 2366
6036,6036,401 1859 3692 1912 829 3105 3418 2470 494 1946
6037,6037,1375 1747 2664 133 1439 12 1379 1102 2502 1296
6038,6038,1355 3343 2231 2632 1514 3278 450 2305 3605 3139


In [86]:
submission.to_csv('./data/submission_BERT100epoch.csv', index=False)

# То что ниже - не трогал

In [42]:
result_metrics = OfflineMetrics(
    [Recall(TOPK), Precision(TOPK), MAP(TOPK), NDCG(TOPK), MRR(TOPK), HitRate(TOPK)], **init_args
)(recommendations.toPandas(), raw_test_gt)

NameError: name 'OfflineMetrics' is not defined

In [ ]:
metrics_to_df(result_metrics)

### User embeddings

Got 6040 x 300 user embeddings, because among all 12 batches:

11 batches contains 512 samples

1 batch contains 408 left samples

11 * 512 + 408 == 6040

In [ ]:
user_embeddings

In [ ]:
user_embeddings.shape

You can access user embeddings directly with `Bert4RecModel` class

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

core_model = Bert4RecModel(
    tensor_schema,
    num_blocks=2,
    num_heads=4,
    max_len=MAX_SEQ_LEN,
    hidden_size=300,
    dropout=0.5
)
core_model.eval()
core_model = core_model.to(device)

# Get first batch of data
data = next(iter(prediction_dataloader))
tensor_map, padding_mask, tokens_mask = data.features, data.padding_mask, data.tokens_mask

# Ensure everything is on the same device
padding_mask = padding_mask.to(device)
tokens_mask = tokens_mask.to(device)
tensor_map["item_id_seq"] = tensor_map["item_id_seq"].to(device)

# Get user embeddings
user_embeddings_batch = core_model.get_query_embeddings(tensor_map, padding_mask, tokens_mask)
user_embeddings_batch

In [ ]:
user_embeddings_batch.shape

## Example of launching an inference for a single user without using a trainer (in order to speed up)
An example for the production of an online script

Let's assume that the user's sequence consisted of a sequence of items [1, 2, 3, 4, 5].
Сreate a padding mask and tokens mask corresponding to the sequence of items.

It is important to take only the latest MAX_SEQ_LEN or less items.

In [ ]:
item_sequence = torch.arange(1, 5).unsqueeze(0)[:, -MAX_SEQ_LEN:]
padding_mask = torch.ones_like(item_sequence, dtype=torch.bool)
tokens_mask = padding_mask.roll(-1, dims=0)
tokens_mask[-1, ...] = 0
sequence_item_count = item_sequence.shape[1]

### Wrapping created tensors in the Bert4RecPredictionBatch entity

In [ ]:
batch = Bert4RecPredictionBatch(
    query_id=torch.arange(0, item_sequence.shape[0], 1).long(),
    padding_mask=padding_mask,
    features={ITEM_FEATURE_NAME: item_sequence.long()},
    tokens_mask=tokens_mask
)

### Run predict step of the Bert4Rec and get scores from the model

In [ ]:
with torch.no_grad():
    scores = best_model.predict_step(batch, 0)
scores

### Getting five items with the highest score

In [ ]:
torch.topk(scores, k=5).indices